# 코로나
* fbprophet 설치 : https://www.python2.net/questions-13349.htm
* https://www.egbiz.or.kr/prjCategory/a/m/selectPrjView.do?prjDegreeId=PD000000019032
* https://zzsza.github.io/data/2019/02/06/prophet/

* 지역별로 나눠서, 이상치 제거해서 시계열 돌려보기

In [1]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
from selenium import webdriver
import time
import datetime
from bs4 import BeautifulSoup
import platform
from matplotlib import font_manager, rc
if platform.system() == 'Darwin': # 맥이면
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # 윈도우면
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')
plt.rcParams['axes.unicode_minus'] = False

Importing plotly failed. Interactive plots will not work.


In [2]:
data = pd.read_csv('D:/data/1-3_전체_데이터_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3327 entries, 0 to 3326
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   지역 + 시            3327 non-null   object
 1   기준일(발병일, 확진일 선택)  3327 non-null   object
 2   무증상/조사중           3327 non-null   object
 3   필터(무증상,조사중 제외)    3327 non-null   object
 4   무증상/조사중 기준일       3327 non-null   object
 5   지역                3327 non-null   object
 6   지역 (group)        3327 non-null   object
 7   확진일자              3327 non-null   object
 8   레코드 수             3327 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 234.1+ KB


In [3]:
data.iloc[:,7] = pd.to_datetime(data.iloc[:,7])
a = data.iloc[:,[0,7]]
a.columns = ['지역','확진일']
a.head()

,지역,확진일
0,고양시,2020-01-26
1,평택시,2020-01-27
2,부천시,2020-02-01
3,부천시,2020-02-02
4,수원시,2020-02-02


In [4]:
b = a['지역'].unique()
b

array(['고양시', '평택시', '부천시', '수원시', '구리시', '시흥시', '김포시', '이천시', '포천시',
       '안양시', '용인시', '파주시', '남양주시', '성남시', '과천시', '광명시', '화성시', '오산시',
       '군포시', '의왕시', '안산시', '광주시', '안성시', '의정부시', '하남시', '동두천시', '양주시',
       '가평군', '양평군', '고양일산동구시', '여주시', '연천군'], dtype=object)

In [5]:
cccc = pd.DataFrame(a[a['지역'] == b[0]]['확진일'].value_counts()).sort_index()
cccc.reset_index(inplace = True)
cccc

,index,확진일
0,2020-01-26,1
1,2020-02-10,1
2,2020-02-27,1
3,2020-03-01,1
4,2020-03-03,1
...,...,...
79,2020-08-27,8
80,2020-08-28,5
81,2020-08-29,6
82,2020-08-30,11


In [6]:
for i in range(len(b)): 
    globals()['a_%s' % b[i]] = pd.DataFrame(a[a['지역'] == b[i]]['확진일'].value_counts()).sort_index()
    globals()['a_%s' % b[i]].reset_index(inplace = True)
    globals()['a_%s' % b[i]].columns = ['ds','y']
    globals()['a_%s' % b[i]]['floor'] = 0
    globals()['a_%s' % b[i]]['cap'] = 100

In [7]:
for i in range(len(b)):
    globals()['m1_%s' % b[i]] = Prophet(growth = 'logistic')
    globals()['m1_%s' % b[i]].fit(globals()['a_%s' % b[i]])
    globals()['future1_%s' % b[i]] = globals()['m1_%s' % b[i]].make_future_dataframe(periods = 30)
    globals()['future1_%s' % b[i]]['floor'] = 0
    globals()['future1_%s' % b[i]]['cap'] = 100
    globals()['forecast1_%s' % b[i]] = globals()['m1_%s' % b[i]].predict(globals()['future1_%s' % b[i]])

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disab

In [8]:
forecast1_고양시

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-26,0.021277,100,0,-2.955210,2.747675,0.021277,0.021277,-0.089014,-0.089014,-0.089014,-0.089014,-0.089014,-0.089014,0.0,0.0,0.0,-0.067736
1,2020-02-10,0.032418,100,0,-3.603242,1.778510,0.032418,0.032418,-1.093072,-1.093072,-1.093072,-1.093072,-1.093072,-1.093072,0.0,0.0,0.0,-1.060654
2,2020-02-27,0.052242,100,0,-2.342413,2.804887,0.052242,0.052242,0.251509,0.251509,0.251509,0.251509,0.251509,0.251509,0.0,0.0,0.0,0.303751
3,2020-03-01,0.056831,100,0,-2.854764,2.601577,0.056831,0.056831,-0.089014,-0.089014,-0.089014,-0.089014,-0.089014,-0.089014,0.0,0.0,0.0,-0.032183
4,2020-03-03,0.060112,100,0,-2.815527,2.726030,0.060112,0.060112,-0.073035,-0.073035,-0.073035,-0.073035,-0.073035,-0.073035,0.0,0.0,0.0,-0.012923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,2020-09-26,16.748297,100,0,15.160805,20.531457,16.748297,16.748297,1.041173,1.041173,1.041173,1.041173,1.041173,1.041173,0.0,0.0,0.0,17.789471
110,2020-09-27,17.143486,100,0,14.144154,19.720950,17.143486,17.143486,-0.089014,-0.089014,-0.089014,-0.089014,-0.089014,-0.089014,0.0,0.0,0.0,17.054473
111,2020-09-28,17.546035,100,0,13.715012,19.054643,17.546035,17.546035,-1.093072,-1.093072,-1.093072,-1.093072,-1.093072,-1.093072,0.0,0.0,0.0,16.452963
112,2020-09-29,17.955988,100,0,15.099704,20.672198,17.955988,17.955988,-0.073035,-0.073035,-0.073035,-0.073035,-0.073035,-0.073035,0.0,0.0,0.0,17.882952


In [9]:
for i in range(len(b)):
    globals()['m2_%s' % b[i]] = Prophet(growth = 'logistic')
    globals()['m2_%s' % b[i]].fit(globals()['a_%s' % b[i]])
    globals()['future2_%s' % b[i]] = globals()['m2_%s' % b[i]].make_future_dataframe(periods = 30)
    globals()['future2_%s' % b[i]]['floor'] = 0
    globals()['future2_%s' % b[i]]['cap'] = 100
    globals()['forecast2_%s' % b[i]] = globals()['m2_%s' % b[i]].predict(globals()['future2_%s' % b[i]])

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [10]:
for i in range(len(b)):
    globals()['m3_%s' % b[i]] = Prophet(growth = 'logistic')
    globals()['m3_%s' % b[i]].fit(globals()['a_%s' % b[i]])
    globals()['future3_%s' % b[i]] = globals()['m3_%s' % b[i]].make_future_dataframe(periods = 30)
    globals()['future3_%s' % b[i]]['floor'] = 0
    globals()['future3_%s' % b[i]]['cap'] = 100
    globals()['forecast3_%s' % b[i]] = globals()['m3_%s' % b[i]].predict(globals()['future3_%s' % b[i]])

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso